<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aerospike-Java-Client-–-Reading-and-Updating-Lists" data-toc-modified-id="Aerospike-Java-Client-–-Reading-and-Updating-Lists-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aerospike Java Client – Reading and Updating Lists</a></span><ul class="toc-item"><li><span><a href="#Notebook-Setup" data-toc-modified-id="Notebook-Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Notebook Setup</a></span><ul class="toc-item"><li><span><a href="#Import-Jupyter-Java-Integration" data-toc-modified-id="Import-Jupyter-Java-Integration-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Import Jupyter Java Integration</a></span></li><li><span><a href="#Start-Aerospike" data-toc-modified-id="Start-Aerospike-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Start Aerospike</a></span></li><li><span><a href="#Download-the-Aerospike-Java-Client" data-toc-modified-id="Download-the-Aerospike-Java-Client-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Download the Aerospike Java Client</a></span></li><li><span><a href="#Start-the-Aerospike-Java-Client-and-Connect" data-toc-modified-id="Start-the-Aerospike-Java-Client-and-Connect-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Start the Aerospike Java Client and Connect</a></span></li></ul></li><li><span><a href="#CREATE-Lists-in-the-Database-and-READ-List-Records" data-toc-modified-id="CREATE-Lists-in-the-Database-and-READ-List-Records-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>CREATE Lists in the Database and READ List Records</a></span></li><li><span><a href="#READING-Lists-and-List-Metrics-From-the-Server" data-toc-modified-id="READING-Lists-and-List-Metrics-From-the-Server-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>READING Lists and List Metrics From the Server</a></span></li><li><span><a href="#UPDATE-Lists-on-the-Server" data-toc-modified-id="UPDATE-Lists-on-the-Server-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>UPDATE Lists on the Server</a></span></li><li><span><a href="#Sorting-the-Lists" data-toc-modified-id="Sorting-the-Lists-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Sorting the Lists</a></span></li><li><span><a href="#DELETING-the-Records-and-Closing-Server-Connection" data-toc-modified-id="DELETING-the-Records-and-Closing-Server-Connection-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>DELETING the Records and Closing Server Connection</a></span></li><li><span><a href="#Aerospike-Does-Lists" data-toc-modified-id="Aerospike-Does-Lists-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Aerospike Does Lists</a></span></li><li><span><a href="#What's-Next?" data-toc-modified-id="What's-Next?-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>What's Next?</a></span><ul class="toc-item"><li><span><a href="#Next-Steps" data-toc-modified-id="Next-Steps-1.8.1"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>Next Steps</a></span></li><li><span><a href="#Additional-Resources" data-toc-modified-id="Additional-Resources-1.8.2"><span class="toc-item-num">1.8.2&nbsp;&nbsp;</span>Additional Resources</a></span></li></ul></li></ul></li></ul></div>

# Aerospike Java Client – Reading and Updating Lists

This notebook demonstrates Java Aerospike CRUD operations (Create, Read, Update, Delete) for lists of data, focusing on server-side **read** and **update** operations, including **sort**. 

This [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/examples_index.html) requires the Aerospike database running locally with Java kernel and Aerospike Java Client. To create a Docker container that satisfies the requirements and holds a copy of these notebooks, visit the [Aerospike Notebooks Repo](https://github.com/aerospike-examples/interactive-notebooks).

## Notebook Setup 

Run these first to initialize Jupyter, download the Java Client, and make sure the Aerospike Database is running.

### Import Jupyter Java Integration 

Make it easier to work with Java in Jupyter.

In [11]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;

IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Start Aerospike

Ensure Aerospike database is running locally.

In [10]:
%sh asd

### Download the Aerospike Java Client

Ask Maven to download and install the project object model (POM) of the Aerospike Java Client.

In [12]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

### Start the Aerospike Java Client and Connect

Create an instance of the Aerospike Java Client, and connect to the demo cluster.

The default cluster location for the Docker container is *localhost* port *3000*. If your cluster is not running on your local machine, modify *localhost* and *3000* to the values for your Aerospike cluster.

In [3]:
import com.aerospike.client.AerospikeClient;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


## CREATE Lists in the Database and READ List Records

1. Create 10 **Keys** containing lists numbered 1 - 10. 
2. Create a String list and an Integer list.
3. Upload the lists into **Bins** named *liststrbin*  and *listintbin*. Place them in the *test* namespace in the *listset1* set.
4. Print the data.
 
A **Namespace** is like a database in Aerospike. A **Set** is like a database table in Aerospike. A **Key** is a record in the Aerospike database. A **Bin** is a field in the database record.

In [4]:
import java.util.ArrayList;
import java.util.Arrays;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;

String listNamespace = "test";
String listSet = "listset1";
String listStrBin = "liststrbin";
String listIntBin = "listintbin";

int numRecords = 10;

String names[] = {"Annette", "Bharat", "Chenguang", "Darrell", "Eva", 
                "Francois", "Geri", "Helen", "Ian", "Javier"};

String fruits[] = {"Pineapple", "Nectarine", "Avocado", "Pear", "Mangosteen", 
                "Date", "Banana", "Orange", "Tomato", "Durian"};

String colors[] = {"Aquamarine", "Mauve", "Snowflake", "Salmon", "Khaki", 
                "Black", "Darkslategray", "Royalblue", "White", "Rainbow"};

ClientPolicy clientPolicy = new ClientPolicy();
for (int keyNum = 0; keyNum < numRecords; keyNum++) {

    ArrayList<String> listStr = new ArrayList<String>();
    listStr.add(names[keyNum]);
    listStr.add(fruits[keyNum]);
    listStr.add(colors[keyNum]);

    ArrayList<Integer> listInt = new ArrayList<Integer>();
    listInt.add(keyNum);
    listInt.add(numRecords-keyNum);
    listInt.add(1+keyNum);
    listInt.add(2*keyNum);
    listInt.add(keyNum*keyNum);

    Key key = new Key(listNamespace, listSet, keyNum);
    Bin bin1 = new Bin(listStrBin, listStr);
    Bin bin2 = new Bin(listIntBin, listInt);
    client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
    
    System.out.println("Key: " + keyNum + ", listStr: " + listStr + ", listInt: " + listInt );
}

Key: 0, listStr: [Annette, Pineapple, Aquamarine], listInt: [0, 10, 1, 0, 0]
Key: 1, listStr: [Bharat, Nectarine, Mauve], listInt: [1, 9, 2, 2, 1]
Key: 2, listStr: [Chenguang, Avocado, Snowflake], listInt: [2, 8, 3, 4, 4]
Key: 3, listStr: [Darrell, Pear, Salmon], listInt: [3, 7, 4, 6, 9]
Key: 4, listStr: [Eva, Mangosteen, Khaki], listInt: [4, 6, 5, 8, 16]
Key: 5, listStr: [Francois, Date, Black], listInt: [5, 5, 6, 10, 25]
Key: 6, listStr: [Geri, Banana, Darkslategray], listInt: [6, 4, 7, 12, 36]
Key: 7, listStr: [Helen, Orange, Royalblue], listInt: [7, 3, 8, 14, 49]
Key: 8, listStr: [Ian, Tomato, White], listInt: [8, 2, 9, 16, 64]
Key: 9, listStr: [Javier, Durian, Rainbow], listInt: [9, 1, 10, 18, 81]


## READING Lists and List Metrics From the Server

Now that the lists are in Aerospike, the client can return full or partial lists.
1. Pull each record from Aerospike.
   * For string lists:
      * Pull the last element from the list.
      * Pull the highest rank item.
   * For the integer lists:
      * Pull numbers between 4 and 20.
      * Pull the 2nd and 3rd ranked items.
4. Print the data.

Note that none of the records were modified by these ops.

In [5]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Operation;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.CTX;
import com.aerospike.client.cdt.ListOperation;

int last = -1;
int highestRank = 2;
int lowerBound = 4;
int upperBound = 21;
int secondRank = 1;  // Rank is 0th based, so 1 is the 2nd rank item.
int rangeRankSize = 2;
int returnTheValues = 7;

for (int keyNum = 0; keyNum < numRecords; keyNum++) {

    Key key = new Key(listNamespace, listSet, keyNum);
    Record record = client.get(null, key);
    Record lastString = client.operate(client.writePolicyDefault, key, 
        ListOperation.get(listStrBin, last)
        );
    Record highestRankString = client.operate(client.writePolicyDefault, key, 
        ListOperation.getByRank(listStrBin, highestRank, returnTheValues)
        );
    Record between4And20 = client.operate(client.writePolicyDefault, key, 
        ListOperation.getByValueRange(listIntBin, Value.get(lowerBound), Value.get(upperBound), returnTheValues)
        );
    Record rank1And2 = client.operate(client.writePolicyDefault, key, 
        ListOperation.getByRankRange(listIntBin, secondRank, rangeRankSize, returnTheValues)
        );
        
    System.out.println("Full record: " + record);
    System.out.println("The last string: " + lastString.getValue(listStrBin));
    System.out.println("The highest rank string: " + highestRankString.getValue(listStrBin));
    System.out.println("The integers between 4 and 20: " + between4And20.getValue(listIntBin));
    System.out.println("The 2nd and 3rd ranked integers: " + rank1And2.getValue(listIntBin));    
    System.out.println();
}

Full record: (gen:27),(exp:350713513),(bins:(liststrbin:[Annette, Pineapple, Aquamarine]),(listintbin:[0, 10, 1, 0, 0]))
The last string: Aquamarine
The highest rank string: Pineapple
The integers between 4 and 20: [10]
The 2nd and 3rd ranked integers: [0, 0]

Full record: (gen:27),(exp:350713513),(bins:(liststrbin:[Bharat, Nectarine, Mauve]),(listintbin:[1, 9, 2, 2, 1]))
The last string: Mauve
The highest rank string: Nectarine
The integers between 4 and 20: [9]
The 2nd and 3rd ranked integers: [2, 1]

Full record: (gen:27),(exp:350713513),(bins:(liststrbin:[Chenguang, Avocado, Snowflake]),(listintbin:[2, 8, 3, 4, 4]))
The last string: Snowflake
The highest rank string: Snowflake
The integers between 4 and 20: [8, 4, 4]
The 2nd and 3rd ranked integers: [4, 3]

Full record: (gen:27),(exp:350713513),(bins:(liststrbin:[Darrell, Pear, Salmon]),(listintbin:[3, 7, 4, 6, 9]))
The last string: Salmon
The highest rank string: Salmon
The integers between 4 and 20: [7, 4, 6, 9]
The 2nd and 3rd r

## UPDATE Lists on the Server 

Aerospike's [list operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html) modify data in the Aerospike database. Here are some examples:

1. Print the record.
2. Modify the string list:
   * Insert a Fish string into the second-from-last position. 
   * Remove the Name. 
3. Modify the integer list:
   * Append a power of 5 to the end the list. 
   * Increment the 4th integer in each list by 11.
4. Print the final record.

In [6]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Operation;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.CTX;
import com.aerospike.client.cdt.ListOperation;

String Fishes[] = {"Koi", "Tuna", "Stingray", "Arowana", "Mackerel",
                "Needlefish", "Sardine", "Angelfish", "Cod", "Mako"};
int fishIndex = -1;
int nameIndex = 0;
int newMult = 5;
int incNum = 11;
int incIndex = 3;

for (int keyNum = 0; keyNum < numRecords; keyNum++) {

    Key key = new Key(listNamespace, listSet, keyNum);
    Record origRecord = client.get(null, key);
    System.out.println("Key:" + keyNum + " Before – " + " listStr: " + origRecord.getValue(listStrBin) + ", listInt: " + origRecord.getValue(listIntBin) );
    origRecord = client.operate(client.writePolicyDefault, key, 
        ListOperation.insert(listStrBin, fishIndex, Value.get(Fishes[keyNum])),
        ListOperation.remove(listStrBin, nameIndex),
        ListOperation.append(listIntBin, Value.get(incNum*keyNum)),
        ListOperation.increment(listIntBin, incIndex, Value.get(incNum))
        );
    Record finalRecord = client.get(null, key);
    System.out.println("      After  – " + " listStr: " + finalRecord.getValue(listStrBin) + ", listInt: " + finalRecord.getValue(listIntBin) );
}

Key:0 Before –  listStr: [Annette, Pineapple, Aquamarine], listInt: [0, 10, 1, 0, 0]
      After  –  listStr: [Pineapple, Koi, Aquamarine], listInt: [0, 10, 1, 11, 0, 0]
Key:1 Before –  listStr: [Bharat, Nectarine, Mauve], listInt: [1, 9, 2, 2, 1]
      After  –  listStr: [Nectarine, Tuna, Mauve], listInt: [1, 9, 2, 13, 1, 11]
Key:2 Before –  listStr: [Chenguang, Avocado, Snowflake], listInt: [2, 8, 3, 4, 4]
      After  –  listStr: [Avocado, Stingray, Snowflake], listInt: [2, 8, 3, 15, 4, 22]
Key:3 Before –  listStr: [Darrell, Pear, Salmon], listInt: [3, 7, 4, 6, 9]
      After  –  listStr: [Pear, Arowana, Salmon], listInt: [3, 7, 4, 17, 9, 33]
Key:4 Before –  listStr: [Eva, Mangosteen, Khaki], listInt: [4, 6, 5, 8, 16]
      After  –  listStr: [Mangosteen, Mackerel, Khaki], listInt: [4, 6, 5, 19, 16, 44]
Key:5 Before –  listStr: [Francois, Date, Black], listInt: [5, 5, 6, 10, 25]
      After  –  listStr: [Date, Needlefish, Black], listInt: [5, 5, 6, 21, 25, 55]
Key:6 Before –  listSt

## Sorting the Lists

Aerospike also sorts and rermoves duplicates from lists.
1. Print both lists.
2. Sort both lists and remove the duplicate integers.
3. Print the sorted lists.

In [8]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Operation;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.CTX;
import com.aerospike.client.cdt.ListOperation;

int dropDuplicates = 2;

for (int keyNum = 0; keyNum < numRecords; keyNum++) {

    Key key = new Key(listNamespace, listSet, keyNum);
    Record origRecord = client.get(null, key);
    System.out.println("Key:" + keyNum + " Unsorted – " + " listStr: " + origRecord.getValue(listStrBin) + ", listInt: " + origRecord.getValue(listIntBin) );
    origRecord = client.operate(client.writePolicyDefault, key, 
        ListOperation.sort(listStrBin, dropDuplicates),
        ListOperation.sort(listIntBin, dropDuplicates)
        );
    Record finalRecord = client.get(null, key);
    System.out.println("        Sorted – " + " listStr: " + finalRecord.getValue(listStrBin) + ", listInt: " + finalRecord.getValue(listIntBin) );
}

Key:0 Unsorted –  listStr: [Aquamarine, Koi, Pineapple], listInt: [0, 1, 10, 11]
        Sorted –  listStr: [Aquamarine, Koi, Pineapple], listInt: [0, 1, 10, 11]
Key:1 Unsorted –  listStr: [Mauve, Nectarine, Tuna], listInt: [1, 2, 9, 11, 13]
        Sorted –  listStr: [Mauve, Nectarine, Tuna], listInt: [1, 2, 9, 11, 13]
Key:2 Unsorted –  listStr: [Avocado, Snowflake, Stingray], listInt: [2, 3, 4, 8, 15, 22]
        Sorted –  listStr: [Avocado, Snowflake, Stingray], listInt: [2, 3, 4, 8, 15, 22]
Key:3 Unsorted –  listStr: [Arowana, Pear, Salmon], listInt: [3, 4, 7, 9, 17, 33]
        Sorted –  listStr: [Arowana, Pear, Salmon], listInt: [3, 4, 7, 9, 17, 33]
Key:4 Unsorted –  listStr: [Khaki, Mackerel, Mangosteen], listInt: [4, 5, 6, 16, 19, 44]
        Sorted –  listStr: [Khaki, Mackerel, Mangosteen], listInt: [4, 5, 6, 16, 19, 44]
Key:5 Unsorted –  listStr: [Black, Date, Needlefish], listInt: [5, 6, 21, 25, 55]
        Sorted –  listStr: [Black, Date, Needlefish], listInt: [5, 6, 21, 25

## DELETING the Records and Closing Server Connection

1. Use the **asinfo** administration tool to dump the set containing our list data.
2. Close the client's connection to the Aerospike cluster.

In [ ]:
%sh asinfo -v "truncate:namespace=test;set=listset1;"
client.close();
System.out.println("Index dropped and server connection closed.");

## Aerospike Does Lists

Aerospike database and its Java Client are up to the task of working with your lists.

## What's Next?

Want to learn more?





### Next Steps

Have questions? Don't hesitate to reach out if you have additional questions about working with lists at https://discuss.aerospike.com/.

Want to check out other Java notebooks?
1. [Hello, World](hello_world.ipynb)
2. [Aerospike Query and UDF](query_udf.ipynb)
3. [Simple Put Get Example](SimplePutGetExample.ipynb)
4. [Working with Twitter Data](tweetaspike.ipynb)

### Additional Resources

Here are some links to learn more about this demo:
* [The Aerospike Java Client](https://github.com/aerospike/aerospike-client-java) 
* [Aerospike's List Operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html)
* [Aerospike Database Architecture](https://www.aerospike.com/docs/architecture/index.html)